# Scrap Kiwilimon

In this file, we'll be downloading and formatting many reciepes from <a href="https://www.kiwilimon.com/" target="_blank">kiwilimon.com<a/>. First try, let's pray 🙏

In [3]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cannot find .env file


In [1]:
# All we need to import

import os
import time
from urllib.parse import urljoin, urlparse

from playwright.async_api import async_playwright

ModuleNotFoundError: No module named 'playwright'

In [ ]:
host = 'https://www.kiwilimon.com'
base_dir = 'data/raw'
bs_parser = 'html.parser'